In [4]:
import pandas as pd
import json
#import glob
#import pickle
from gensim.models import KeyedVectors
fname="C:/data/GoogleNews-vectors-negative300.bin"
model_w2v = KeyedVectors.load_word2vec_format(fname, binary=True)
print("loaded")

C:\Users\Mathieu Lemay\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


loaded


In [2]:
df = pd.read_csv('scrapedSchools.csv')

In [6]:
#load csv into pandas
import pandas as pd
import numpy as np

def sent2vec(sentence):
    sentence=sentence.replace(",",'').replace(".",'')
    sentenceVec=np.zeros(300)
    wordCount=0.0
    for word in sentence.split(" "):
        if word in model_w2v:
            sentenceVec=sentenceVec+model_w2v[word]
            wordCount+=1
    if wordCount>0:
        sentenceVec=sentenceVec/wordCount
    sentenceRatio = wordCount/max(len(sentence.split(" ")),1)
    return sentenceRatio, sentenceVec

def docToSentVecs(posting):
    sentences = posting.split('.')
    sentenceVecs=[]
    sentenceRatios=[]
    for sentence in sentences:
        sentenceRatio, sentenceVec = sent2vec(sentence)
#         print(sentenceRatio, sentenceVec)
        sentenceRatios.append(sentenceRatio)
        sentenceVecs.append(sentenceVec)
    return sentenceVecs, sentenceRatios


sentences=[]
y=[]



In [7]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 5 columns):
Unnamed: 0    329 non-null int64
link          329 non-null object
linkTitle     329 non-null object
schoolLink    329 non-null object
pageText      291 non-null object
dtypes: int64(1), object(4)
memory usage: 12.9+ KB


,Unnamed: 0,link,linkTitle,schoolLink,pageText
0,0,http://www.athabascau.ca/syllabi/comp/comp683.php,Introduction to Learning and Knowledge Analyti...,www.athabascau.ca,Skip To Co...
1,1,http://www.athabascau.ca/syllabi/comp/comp682.php,Data Mining : Computer Science 682 : Courses :...,www.athabascau.ca,Skip To Co...
2,2,http://www.athabascau.ca/programs/summary/bach...,Bachelor of Science in Computing and Informati...,www.athabascau.ca,Skip To Con...
3,3,http://www.athabascau.ca/syllabi/comp/comp602.php,Enterprise Information Management : Computer S...,www.athabascau.ca,Skip To Co...
4,4,http://www.athabascau.ca/programs/program-list...,Programs : Athabasca University,www.athabascau.ca,Skip To Co...


In [9]:
import re

x1 = []
x2 = []
y =  []
for index, row in df.iterrows():
    posting=row['pageText']
    sentenceVecs, sentenceRatios = docToSentVecs(str(posting))
    sr=0
    sv=0
    for i in range(len(sentenceRatios)):
        sv += sentenceVecs[i]
        sr += sentenceRatios[i]
    sv=sv/len(sentenceRatios)
    sr=sr/len(sentenceRatios)
    #print(sr,sv)

    
    if sr > 0.4:
        #1) cluster titles with semantic embedding on titles
        x1.append(sv)

        y.append(str(row['linkTitle']))
    
        title = re.sub('[^0-9a-zA-Z]+', ' ', str(row['schoolLink']))
        x2.append(title)


# Load job posts
Read the text from various job posts.

In [18]:
import json
with open('jobs.json', 'r') as f:
    data = json.loads("".join(f.read().replace('\n', ' ')))
#data
jobVecs, jobRatios = docToSentVecs(str(posting))
sr=0
sv=0
for i in range(len(sentenceRatios)):
    sv += sentenceVecs[i]
    sr += sentenceRatios[i]
sv=sv/len(sentenceRatios)
sr=sr/len(sentenceRatios)

jobVector = sv

In [19]:
jobVector

array([ 0.03057659,  0.0046178 ,  0.0243139 , -0.04004698, -0.00238848,
        0.04073752,  0.01559981, -0.1074383 , -0.01089411, -0.07410535,
       -0.01271699, -0.02588911, -0.00772294,  0.0792416 , -0.09609888,
        0.06152489,  0.12760453,  0.07929145,  0.01354925,  0.01710254,
       -0.00639256, -0.01681657,  0.05186294,  0.01059497,  0.05055026,
       -0.06912944, -0.03384207, -0.00797134, -0.04170668, -0.00478327,
       -0.08242918, -0.04840085,  0.05433315, -0.09028914,  0.07578022,
       -0.1169096 ,  0.00916335,  0.12447113, -0.0398422 , -0.03094409,
       -0.01569089,  0.02503588, -0.0839896 ,  0.0208138 , -0.07101329,
       -0.07174982, -0.03315007, -0.02674401,  0.04387827, -0.00907085,
       -0.07707234,  0.01860062,  0.01950426, -0.07240125,  0.01838285,
       -0.03028236, -0.075233  , -0.05182953,  0.01049526, -0.12066938,
       -0.09014301, -0.03525632, -0.02332067,  0.03079029,  0.03398812,
       -0.04027197, -0.06592482,  0.04912653,  0.02259519, -0.00

In [40]:
from sklearn.metrics.pairwise import cosine_similarity
data = []
for i in range(len(x1)):
    #print(i[1])
    distance = np.average(cosine_similarity(np.array(x1[i]), np.array(jobVector)), axis=0)
    print(distance)
    print(y[i])
    data.append({"distance":distance[0], "title":y[i]})

distdf = pd.DataFrame(data)    

C:\Users\Mathieu Lemay\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Mathieu Lemay\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Mathieu Lemay\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single s

[ 0.29500927]
Introduction to Learning and Knowledge Analytics : Computer ...
[ 0.4039255]
Data Mining : Computer Science 682 : Courses : Athabasca University
[ 0.29444242]
Bachelor of Science in Computing and Information Systems ...
[ 0.30402616]
Enterprise Information Management : Computer Science 602 ...
[ 0.27956409]
Programs : Athabasca University
[ 0.30468625]
Data Structures and Algorithms : Computer Science 272 : Courses ...
[ 0.30602701]
University Certificate in Computing and Information Systems ...
[ 0.33067801]
Bachelor of General Studies : Programs : Athabasca University
[ 0.37223031]
Introduction to Database Management : Computer Science 378 ...
[ 0.39517185]
Enterprise Information Security : Computer Science 660 : Courses ...
[ 0.19234919]
Objects First - Augustana Campus
[ 0.41902803]
DeVry University: Online College & University
[ 0.4418198]
Data Science | University of Calgary |
[ 0.31413509]
Minor in Data Science
[ 0.44593456]
Research
[ 0.5178597]
Contact Us
[ 0.314

C:\Users\Mathieu Lemay\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Mathieu Lemay\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Mathieu Lemay\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single s

In [46]:
distdf.sort_values('distance').head(20)

,distance,title
286,0.042174,Saint Mary's University | M.Sc. in Computing &...
129,0.042174,Parksville?Qualicum Centre | Vancouver Island ...
109,0.042174,Douglas College aims to help end stigma surrou...
110,0.042174,Governance Douglas College - Douglas College
111,0.042174,Fitness Centre Information - Student Life - Do...
287,0.042174,"Saint Mary's University | Videos, M.Sc. in Com..."
288,0.042174,Saint Mary's University | Computing Science Un...
289,0.042174,Graduate Profiles - Saint Mary's University |
291,0.042174,Saint Mary's University | eLearning Centre - F...
106,0.042174,Geography - Programs - Douglas College
